# Import
---

In [1]:
# suppress warnings
import warnings
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

warnings.filterwarnings('ignore')

# reload all modules every time before executing the Python code
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import os
import sys
import ast

import io
from io import BytesIO
import boto3

import pandas as pd
import numpy as np
from nltk.util import ngrams
from collections import Counter
import utils.s3helpers as s3
from datasets import load_dataset
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.async_inference import AsyncInferenceConfig
import sagemaker

print(f'default sys.path: {sys.path}')
# Probably not needed for pycharm but needed for vscode -----------------------------------
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
sys.path.append(PROJ_ROOT)
print(f'Project root: {PROJ_ROOT}')
print("\n")
# Probably not needed for pycharm but needed for vscode -----------------------------------

from pathlib import Path
import re
import json
from tqdm.auto import tqdm
import ast

from huggingface_hub import login

hf_token = ""
login(token=hf_token)
print("\n")

# from vllm import SamplingParams
# from vllm import LLM
# import multiprocessing
# multiprocessing.set_start_method('spawn', force=True)
# from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
import torch
import torch.multiprocessing as mp
# Check if CUDA is available
print("\n")
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
# If not, check if MPS is available
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS")
# If neither CUDA nor MPS is available, use CPU
else:
    device = torch.device("cpu")
    print("Using CPU")

# Test the device
x = torch.ones(1, device=device)
print(x)

def print_gpu_memory():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            device = torch.cuda.device(i)
            props = torch.cuda.get_device_properties(device)
            total_memory = props.total_memory / 1e9  # Convert to GB
            allocated_memory = torch.cuda.memory_allocated(device) / 1e9  # Convert to GB
            reserved_memory = torch.cuda.memory_reserved(device) / 1e9  # Convert to GB
            
            print(f"GPU {i}: {props.name}")
            print(f"  Total Memory: {total_memory:.2f} GB")
            print(f"  Allocated Memory: {allocated_memory:.2f} GB")
            print(f"  Reserved Memory: {reserved_memory:.2f} GB")
            print(f"  Free Memory: {total_memory - reserved_memory:.2f} GB")
            print()
    else:
        print("CUDA is not available. No GPU detected.")

print_gpu_memory()

# Check data in S3 buckets
print("\n")
s3.list_s3_buckets()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
default sys.path: ['/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/opt/conda/lib/python3.10/site-packages/setuptools/_vendor']
Project root: /home/sagemaker-user


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/sagemaker-user/.cache/huggingface/token
Login successful




2024-12-11 00:58:15.762509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 00:58:15.762550: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 00:58:15.762558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 00:58:15.768715: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.




Using CUDA
tensor([1.], device='cuda:0')
GPU 0: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 1: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 2: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 3: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 4: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 5: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 6: NVIDIA A100-SXM4-80GB
  Total Memory: 84.97 GB
  Allocated Memory: 0.00 GB
  Reserved Memory: 0.00 GB
  Free Memory: 84.97 GB

GPU 7: NVIDIA A100-

---
# Combine dfs
---

In [ ]:
# mistralai/Mistral-7B-Instruct-v0.3 - meta-llama/Llama-3.1-8B-Instruct","HuggingFaceH4/zephyr-7b-beta","Qwen/Qwen2.5-7B-Instruct"]

In [2]:
data_prefix = "Persona/output/persona_distribution/zephyr-7b-beta/right_authoritarian_personas/sub_dfs_stanced"


combined_df = None
for i in tqdm(range(20)):
    df = s3.read_s3_parquet(f"{data_prefix}/df_b{i}_p10000.pqt")
    if combined_df is None:
        combined_df = df
    else:
        combined_df = pd.concat([combined_df, df], ignore_index=True)

print(combined_df.shape)

  0%|          | 0/20 [00:00<?, ?it/s]

(12400000, 6)


In [3]:
print(f"Number of unique persona_ids: {combined_df['persona_id'].nunique()}")

Number of unique persona_ids: 200000


In [20]:
display(combined_df.tail(10))

,statement_id,statement,persona_id,persona,prompt,response
12399990,52,You cannot be moral without being religious.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399991,53,Charity is better than social security as a me...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399992,54,Some people are naturally unlucky.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399993,55,It is important that my child's school instill...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399994,56,Sex outside marriage is usually immoral.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399995,57,"A same sex couple in a stable, loving relation...",199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Strongly agree
12399996,58,"Pornography, depicting consenting adults, shou...",199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Agree
12399997,59,What goes on in a private bedroom between cons...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399998,60,No one can feel naturally homosexual.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree
12399999,61,These days openness about sex has gone too far.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree


In [4]:
response_mapping = {
    'Strongly disagree': 0,
    'Disagree': 1,
    'Agree': 2,
    'Strongly agree': 3
}

combined_df['int_stance'] = combined_df['response'].map(response_mapping)
display(combined_df.tail(10))


,statement_id,statement,persona_id,persona,prompt,response,int_stance
12399990,52,You cannot be moral without being religious.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399991,53,Charity is better than social security as a me...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399992,54,Some people are naturally unlucky.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399993,55,It is important that my child's school instill...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399994,56,Sex outside marriage is usually immoral.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399995,57,"A same sex couple in a stable, loving relation...",199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399996,58,"Pornography, depicting consenting adults, shou...",199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1
12399997,59,What goes on in a private bedroom between cons...,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Strongly agree,3
12399998,60,No one can feel naturally homosexual.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Strongly disagree,0
12399999,61,These days openness about sex has gone too far.,199999,[TOKEN] backyard grill master who loves to sha...,Your task is to evaluate a statement and provi...,Disagree,1


---
#### Save data
---

In [5]:
output_prefix = "Persona/output/persona_distribution/zephyr-7b-beta/right_authoritarian_personas"

In [6]:
s3.write_s3_parquet(combined_df, f"{output_prefix}/persona_compass.pqt")
print(f"Data saved to: {output_prefix}/persona_compass.pqt")

Data saved to: Persona/output/persona_distribution/zephyr-7b-beta/right_authoritarian_personas/persona_compass.pqt
